In this notebook, we will demonstrate how the newly implemented EEG pipeline can be called from command line. 

It is important to note that cmp _does not_ include preprocessing, so it is expected that you have your data ready to be analyzed. 

For visualization purposes, we are using subject 1 from the "VEPCON" dataset, available at ..., however, note that non-defaced MRIs were used to create the inverse solutions, as defacing creates distortions. 

In [ ]:
import sys
import os
import argparse

import subprocess

# BIDS import
from bids import BIDSLayout

# CMP imports
import cmp.project
from cmp.info import __version__, __copyright__
from cmtklib.util import print_error, print_blue, print_warning

import warnings

# imports for this notebook
import pdb
import numpy as np
import matplotlib.pyplot as plt
import pickle
from cmtklib.bids.io import __nipype_directory__
from IPython.display import SVG, display
import mne
import mne_connectivity as mnec

First, we need to set up out cmp project with user-defined arguments. This is usually done using parser, but we do it manually here for demo purposes.

A call from the CLI looks like this:

In [ ]:
'''connectomemapper3 --participant_label sub-01 --anat_pipeline_config /home/katharina/data/DS003_BIDS/code/ref_anatomical_config.json --eeg_pipeline_config /home/katharina/data/DS003_BIDS/code/ref_mne_eeg_config.json --bids_dir /home/katharina/data/DS003_BIDS --output_dir /home/katharina/data/DS003_BIDS/derivatives'''

In [ ]:
# We have arguments:
bids_dir = '/home/katharina/data/DS003_BIDS'
output_dir = '/home/katharina/data/DS003_BIDS/derivatives'
participant_label = 'sub-01'
anat_pipeline_config = '/home/katharina/data/DS003_BIDS/code/ref_anatomical_config.json'

# we will first demonstrate the MNE-based pipeline
eeg_pipeline_config = '/home/katharina/data/DS003_BIDS/code/ref_mne_eeg_config.json'

The eeg pipeline config .json file contains information that cmp needs to correctly load EEG data and associated information like electrode positions, names of conditions, which parcellation to use, etc. 

In [ ]:
# initialize project
project = cmp.project.ProjectInfo()
project.base_directory = os.path.abspath(bids_dir)
project.output_directory = os.path.abspath(output_dir)
project.subjects = ["{}".format(participant_label)]
project.subject = "{}".format(participant_label)
# anatomical pipeline is always run
project.anat_config_file = os.path.abspath(anat_pipeline_config)
# in our case, we do not have sessions
project.subject_sessions = [""]
project.subject_session = ""
# check for BIDS layout
bids_layout = BIDSLayout(project.base_directory)

In [ ]:
# parts of connectomemapper3.py that are relevant here

# process the anatomical pipeline (e.g. to get Freesurfer derivatives necessary for MNE pipeline)
# this takes a while when done for the first time (several hours for one subject)
anat_pipeline = cmp.project.init_anat_project(project, False)
if anat_pipeline is not None:
    # check if inputs to anatomical pipeline are valid
    anat_valid_inputs = anat_pipeline.check_input(bids_layout, gui=False)
    if anat_valid_inputs:
        print(">> Process anatomical pipeline")
        anat_pipeline.process()
    else:
        print_error("  .. ERROR: Invalid inputs")
        exit_code = 1
anat_valid_outputs, msg = anat_pipeline.check_output()
project.freesurfer_subjects_dir = anat_pipeline.stages['Segmentation'].config.freesurfer_subjects_dir
project.freesurfer_subject_id = anat_pipeline.stages['Segmentation'].config.freesurfer_subject_id

In [ ]:
# process EEG pipeline
project.eeg_config_file = os.path.abspath(eeg_pipeline_config)
if anat_valid_outputs:
    eeg_valid_inputs, eeg_pipeline = cmp.project.init_eeg_project(project, bids_layout, False)
    if eeg_pipeline is not None:
        eeg_pipeline.parcellation_scheme = anat_pipeline.parcellation_scheme
        eeg_pipeline.atlas_info = anat_pipeline.atlas_info

        if eeg_valid_inputs:
            print(">> Process EEG pipeline")
            eeg_pipeline.process()
        else:
            print("  .. ERROR: Invalid inputs")
            exit_code = 1
else:
    print_error(f'  .. ERROR: Invalid anatomical outputs for eeg pipeline')
    print_error(f'{msg}')
    exit_code = 1

Let's have a closer look at the outputs that the EEG pipeline produces.

First of all: Connectomemapper works in such a way that the pipeline is first assembled and only afterwards, it is executed. During the assembly stage, input and output variables are connected and cmp produces a graph that visualizes this. 

In [ ]:
path_to_svg = os.path.join(output_dir,__nipype_directory__,participant_label,'eeg_pipeline','graph.svg')
display(SVG(filename=path_to_svg))

Apart from the input ("datasource") and output ("eeg sinker") nodes that are apart, you can see three blue boxes that represent the stages of the pipeline flow (preparer, loader, inverse solution). Each of the stages, again, has an input and and output node, as well as several nodes representing processing steps. Each processing step has its own "interface" which you can find in cmtklib/interfaces.

In the following, we will go over the interfaces and show what output they produce. 

# Preparing stage

Let's first have a look at the information given via the config file regarding this stage: 

>     "eeg_preparing_stage": {
        "eeg_format": ".set",
        "epochs": "sub-01_task-faces_desc-preproc_eeg.set",
        "invsol_format": "mne-sLORETA",
        "parcellation": {
            "label": "lausanne2008", 
            "desc": "",
            "suffix": "scale1"
        },        
        "number_of_threads": 1,
        "EEG_params": {
            "expe_name": "faces",
            "EEG_event_IDs": {
            	"SCRAMBLED" : 0,
            	"FACES" : 1 
            	},
            "start_t": -0.2, 
            "end_t" : 0.6
        }
    },


The preparing stage has three processing steps: 

- eeglab2fif: reads eeglab data and converts them to MNE format (.fif file extension)
- createsrc: creates the dipole locations along the surface of the brain 
- createbem: creates the boundary element method

**eeglab2fif**

If your data are not already in MNE format (.fif file extension), they have to be read and re-saved. The eeglab2fif interface does this for EEGLAB-format data (.set file extension). The interface produces a file named sub-01_epo.fif in the derivatives/cmp-v3.0.2 folder. Critically, the saved epochs contain a montage, i.e. the sensor locations which have to be supplied in a file names sub-01.xyz inside the subject's EEGLAB derivatives folder (derivatives/eeglab/sub-01/eeg/sub-01.xyz). 

In [ ]:
# Let's have a look at the EEG data
with warnings.catch_warnings(): # suppress some irrelevant warnings coming from mne.read_epochs_eeglab()
    warnings.simplefilter("ignore")
    epochs_eeglab = mne.read_epochs_eeglab(os.path.join(output_dir,'eeglab',\
                                participant_label,'eeg',participant_label+'_task-faces_desc-preproc_eeg.set')) # sub-01_FACES_250HZ_prepd.set

# eeglab2fif removes a baseline and crops the epochs according to parameters start_t and end_t in config file
start_t = -0.2
end_t = 0.6
epochs_eeglab.apply_baseline((start_t,0))
epochs_eeglab.crop(tmin=start_t,tmax=end_t)
evoked_eeglab = epochs_eeglab.average().pick('eeg')

# compare to what eeglab2fif saved
epochs_mne = mne.read_epochs(os.path.join(output_dir,'cmp-v3.0.2',\
                                participant_label,'eeg',participant_label+'_epo.fif'))
evoked_mne = epochs_mne.average().pick('eeg')

In [ ]:
# plot
fig = plt.figure()
plt.rcParams['figure.figsize'] = (15, 10)

_=evoked_mne.plot(time_unit='s')

fig = plt.figure()
plt.rcParams['figure.figsize'] = (15, 10)

_=evoked_eeglab.plot(time_unit='s')

**createsrc**

MNE is able to create volume- and surface-based source spaces, but in our pipeline, we use surface-based only. In order to do this, MNE takes advantage of the Freesurfer-created outputs in the freesurfer-6.0.1 derivatives directory. 

**createbem**

The BEM (boundary element model) is the head model we use, in our case, it is based on the individual's structural MRI and, again, related freesufer derivatives. Its creation consists of two steps. First, the necessary surfaces are extracted using mne.bem.make_watershed_bem: brain, inner skull, outer skull, and outer skin. The surfaces are saved in the subject's freesurfer-directory in a new folder bem/watershed. Second, the model itself is created using mne.make_bem_model and mne.make_bem_solution. In this step, the surfaces and the tissue conductivities between the surfaces are used. 

In [ ]:
# Let's visualize the BEM surfaces and source space
src = mne.read_source_spaces(os.path.join(output_dir,'cmp-v3.0.2',\
                                participant_label,'eeg',participant_label+'_src.fif'))
# plot will appear in separate window
%matplotlib qt 
# lines are the surfaces, pink dots are the sources (dipoles)
_=mne.viz.plot_bem(subject=participant_label, subjects_dir=project.freesurfer_subjects_dir,
                 brain_surfaces='white', src=src, orientation='sagittal')

# loader stage

During the preparer stage, we have told cmp which file extensions and keywords to look for that are going to be used in the actual inverse solution. Using nipype's "BIDS datagrabber", it gathers the necessary inputs in this step. 

# inverse solution stage

The only implemented inverse solution algorithm right now is sLORETA. 

This stage has again three processing steps: 

- createfwd: creates the forward solution (leadfield) from the BEM and the source space
- createcov: creates the noise covariance matrix from the data
- invsol_MNE: creates the actual inverse operator and applies it, resulting in ROI-time courses

**createfwd**

In creating the forward solution, the electrode positions need to be known. Remember, we added them to the MNE epochs during execution of eeglab2fif. However, in this step, it is crucial that those coordinates and the Freesurfer-surfaces used in the creation of the head model and the surface space are aligned. Any necessary transformations area passed to mne.make_forward_solution as the "trans" argument. Since we are using non-defaced MRIs, which are not exactly the same as the ones provided on OpenNeuro, we need this transform: 

```
array([[ 1.   ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  1.   ,  0.   , -0.009],
       [ 0.   ,  0.   ,  1.   ,  0.011],
       [ 0.   ,  0.   ,  0.   ,  1.   ]])
```

The transform has to be provided as sub-01-trans.fif and can be created manually or from a text-file using mne.write_trans. Here we assume that we have this file and can load it. 

In [ ]:
# Let's check the alignment between MRI and electrode positions.
trans = mne.read_trans(os.path.join(output_dir,'cmp-v3.0.2',\
                                participant_label,'eeg',participant_label+'-trans.fif'))
mne.viz.plot_alignment(epochs_mne.info, trans=trans, subject=participant_label,
                       subjects_dir=project.freesurfer_subjects_dir, dig=False,
                       surfaces=dict(head=0.95), coord_frame='mri')

**createcov**

MNE uses an estimate of signal to noise ratio in its creation of the inverse solution. For that, it considers the pre-stimulus period of the EEG recordings.

In [ ]:
# Let's have a look at the noise covariance.
# go back to inline plotting
%matplotlib inline 
noise_cov = mne.read_cov(os.path.join(output_dir,'cmp-v3.0.2',\
                                participant_label,'eeg',participant_label+'_noisecov.fif'))
#fig_cov, fig_spectra = mne.viz.plot_cov(noise_cov, epochs_mne.info)

**invsol_MNE**

Now, everything comes together to create the inverse operator, which is then applied to the EEG data to create source time courses. In the last step, the source time courses are converted to ROI-time courses according to the selected parcellation. 

The outputs that are necessary for this step to work were created in the previous processing steps, namely: 

- the EEG epochs in .fif-format
- the electrode montage
- the head model
- the source point locations
- the forward operator
- the noise covariance

First, the inverse operator is created using mne.minimum_norm.make_inverse_operator. We use the options 

```
loose=1, depth=None, fixed=False
```

This means that we are obtaining full 3-dimensional dipoles whose orientation is not fixed or constrained to be (somewhat) orthogonal to surface; and we are not applying any depth weighting. The solution is written to a file sub-01-inv.fif in the same directory as the other outputs (derivatives/cmp-v3.0.2/sub-01/eeg). 

In a subsequent step in the same interface, this inverse operator is then applied to the epochs (not the evoked time course averaged over trials) using mne.minimum_norm.apply_inverse_epochs. 

The final step performed by this interface and by the EEG pipeline is to use mne.extract_label_time_course to create ROI-time courses according to mne.read_labels_from_annot. As given in the config file, we use "lausanne2008" scale 1, which is the Desikan-atlas. The time courses and the ROI-names are stored in sub-01_rtc_epo.pkl, i.e. a format that is not specific to MNE. 

In [ ]:
# Let's have a look at the time courses.
with open(os.path.join(output_dir,'cmp-v3.0.2',participant_label,'eeg',participant_label+'_rtc_epo.pkl'),'rb') as f:
    rtc_epo = pickle.load(f)
    # for some reason, MNE writes label time courses as lists. convert to numpy array
    #rtc_epo['data'] = np.array(rtc_epo['data'])

In [ ]:
# sort labels to make the time courses look nicer
N = len(rtc_epo['labels'])-2 # two "unknown" regions - do not plot
sorting = list(np.arange(0,N,2))+list(np.arange(1,N,2)) # left and right always alternating
# list of ROI names
labels_list_left = [i.name for i in rtc_epo['labels'][0::2] if i.name!='unknown -lh']
labels_list_right = [i.name for i in rtc_epo['labels'][1::2] if i.name!='unknown -rh']
labels_list = labels_list_left+labels_list_right

In [ ]:
# plot
plt.rcParams['figure.figsize'] = (15, 10)
plt.imshow(np.mean(rtc_epo['data'][:,:-2,:],axis=0)[sorting,:],aspect='auto',extent=[-200,600,0,67],interpolation='None');
plt.xlabel('ms')
plt.ylabel('ROIs')
plt.colorbar()
locs = np.arange(0,N)
_=plt.yticks(locs,labels_list[-1::-1] )  

# Connectivity measures

Of course, the idea of cmp is to provide connectomes! This interface is not implemented yet in the pipeline, but with the ROI-time courses, it is easy to obtain functional connectivity matrices using MNE functions. 



In [ ]:
import pdb
import importlib

sfreq = epochs_mne.info['sfreq']  # the sampling frequency
con_methods = ['pli', 'wpli2_debiased', 'ciplv']

label_ts = rtc_epo['data']

con = mnec.spectral_connectivity(
    label_ts, method=con_methods, mode='multitaper', sfreq=sfreq,
    faverage=True, mt_adaptive=True, n_jobs=1)

# con is a 3D array, get the connectivity for the first (and only) freq. band
# for each method
con_res = dict()
for method, c in zip(con_methods, con):
    con_res[method] = c.get_data(output='dense')[:, :, 0]



In [ ]:
label_names = [label.name for label in rtc_epo['labels']]

lh_labels = [name for name in label_names if name.endswith('lh')]

# Get the y-location of the label
label_ypos = list()
for name in lh_labels:
    idx = label_names.index(name)
    ypos = np.mean(rtc_epo['labels'][idx].pos[:, 1])
    label_ypos.append(ypos)

# Reorder the labels based on their location
lh_labels = [label for (yp, label) in sorted(zip(label_ypos, lh_labels))]

# For the right hemi
rh_labels = [label[:-2] + 'rh' for label in lh_labels]

# Save the plot order and create a circular layout
node_order = list()
node_order.extend(lh_labels[::-1])  # reverse the order
node_order.extend(rh_labels)

node_angles = mnec.viz.circular_layout(label_names, node_order, start_pos=90,
                              group_boundaries=[0, len(label_names) / 2])

# Plot the graph using node colors from the FreeSurfer parcellation. We only
# show the 300 strongest connections.
mnec.viz.plot_connectivity_circle(con_res['wpli2_debiased'], label_names, n_lines=300,
                         node_angles=node_angles, node_colors='r',
                         title='All-to-All Connectivity left-Auditory '
                               'Condition (PLI)')